# LPC55Sxx Secure Firmware Update

This notebook describes how to create secure container for firmware update on LPC55Sxx devices using the SPSDK command line utilities, it is based on the application note https://www.nxp.com/docs/en/application-note/AN12283.pdf. 

### Secure Boot Setup
We need to setup secure boot first. This is better described in "lpc55sx_secure_boot.ipynb" notebook. So, let's execute it first.

In [74]:
%run ./lpc55sxx_secure_boot.ipynb

Created `%!` as an alias for `%execute`.
nxpimage -v utils binary-image convert -i lpcxpresso55s69_led_blinky.axf -f BIN workspace/lpcxpresso55s69_led_blinky.bin
INFO:spsdk.apps.nxpimage:
+--0x0000_0000--lpcxpresso55s69_led_blinky.axf--+
|                 Size: 8.8 kB                  |
|The image loaded from: /Users/macbook-m1/spsdk_|
|test/spsdk/examples/jupyter_examples/lpc55sxx_s|
|  ecure_boot/lpcxpresso55s69_led_blinky.axf .  |
|+--0x0000_0000--Segment 0---------------------+|
||                Size: 8.8 kB                 ||
|+--0x0000_2273--------------------------------+|
+--0x0000_2273----------------------------------+

Success. (Converted file: workspace/lpcxpresso55s69_led_blinky.bin created.)
nxpimage -v mbi get-templates -f lpc55s6x workspace/templates
Skip creating workspace/templates/lpc55s6x_int_xip_plain.yml, this file already exists.
Skip creating workspace/templates/lpc55s6x_int_xip_crc.yml, this file already exists.
Skip creating workspace/templates/lpc55s6x_int_x

### SBKEK generation

SBKEK is key (AES-256) used for encryption of the Secure Binary. We have to prepare it in plaint text and in binary form. We can use nxpimage sb21 get-sbkek tool for key generation.

In [75]:
WORKSPACE = "workspace/"

# we can specify key with -k option. If the key is not specified, random one will be generated.
%! nxpimage sb21 get-sbkek -o $WORKSPACE

SBKEK_PATH = WORKSPACE + "sbkek.txt"
SBKEK_BIN = WORKSPACE + "sbkek.bin"


nxpimage sb21 get-sbkek -o workspace/
SBKEK: 05401b7e5c003b75c69df91999e0fd1eeb3a8cf46686ba9974fdb3c94bc2083d
SBKEK has been stored to: workspace/


### Secure Binary preparation
Secure Binary (SB) is a container in case of LPC55xx in version 2.1. It's symmetrically encrypted using AES-256 and decrypted during boot and digitally signed. The SB configuration file contains the configuration commands that are processed after the SB2 file is loaded in the device. The image location is stated in the "sources" section of the .bd file. The SB key in the text file is used for encryption with the nxpimage command line tool.

In the example below we have prepared simple BD file with one section with two commands - erase and load.

In [76]:
import os


BD_FILE_PATH = WORKSPACE + "signed.bd"
SB2_PATH = WORKSPACE + "output.sb2"


bd = """options {
 flags = 0x8; // for sb2.1 use only 0x8 encrypted + signed
 buildNumber = 0x1;
 productVersion = "1.00.00";
 componentVersion = "1.00.00";
 secureBinaryVersion = "2.1";
}
sources
{
 inputFile = extern(0);
}
section (0)
{
 erase 0..0x00010000;
 load inputFile > 0x0;
}"""

# write configuration to file
with open(BD_FILE_PATH, "w+") as bd_file:
    bd_file.write(bd)   

assert os.path.exists(SBKEK_PATH)
assert os.path.exists(SBKEK_BIN)

%! nxpimage $VERBOSITY sb21 export -k $SBKEK_PATH -c $BD_FILE_PATH -o $SB2_PATH -s $PRIVATE_KEY_PATH -S $ROOT_0_CERT_PATH -S $CHAIN_CERT_0 -R $ROOT_0_CERT_PATH $BIN_OUTPUT_PATH
assert _exit_code == 0

assert os.path.exists(SB2_PATH)


nxpimage -v sb21 export -k workspace/sbkek.txt -c workspace/signed.bd -o workspace/output.sb2 -s workspace/rsa2048_key.pem -S workspace/ROT1_sha256_2048_65537_v3_ca_crt.der -S workspace/IMG1_1_sha256_2048_65537_v3_usr_key.pem -R workspace/ROT1_sha256_2048_65537_v3_ca_crt.der workspace/lpc55s6x_mbi.bin
Success. (Secure binary 2.1: workspace/output.sb2 created.)


## Device preparation

Now it's time to prepare the device (enroll keys, load pfr...). In this example we will use LPCXpresso55S69 Evaluation kit.

First step is to enter ISP mode, this could be achieved by either shorting J10 or by simultaneously pressing ISP button and reset button.

LPCXpresso55S69 supports UART and USB-HID interface for the ISP programming. In the picture below we used UART, if you want to use USB, connect the cable to high speed USB port.

![lpc55s69](img/lpc55s69.png "lpc55s69")


We could use app *nxpdevscan* to check if the device is connected to the PC in ISP mode.



In [77]:
# check if the device is connected and detected by PC
%! nxpdevscan

nxpdevscan
-------- Connected NXP USB Devices --------

USB COMPOSITE DEVICE - NXP SEMICONDUCTOR INC.
Vendor ID: 0x1fc9
Product ID: 0x0021
Path: DevSrvsID:4295541984
Name: LPC55, RT6xx
Serial number: 

-------- Connected NXP UART Devices --------

-------- Connected NXP SIO Devices --------



In [78]:
USB_CONNECTION = "-u lpc55" 
# choose com port or /dev
UART_CONNECTION = "-p com6"

# comment if you want to use UART
CONNECTION = USB_CONNECTION
# CONNECTION = UART_CONNECTION

%! blhost $CONNECTION get-property current-version
assert _exit_code == 0

blhost -u lpc55 get-property current-version
Response status = 0 (0x0) Success.
Response word 1 = 1258487808 (0x4b030000)
Current Version = K3.0.0


### Key enrollment

Key enrollment should be done only once in the device life cycle. We have to write KEK key that will be used for SB decryption and we will generate PUF keys for PRINCE encryption.

To detect if key provisioning was enrolled issue blhost command *read-memory 0x0009E600 4*. If the output is "95 95 95 95" it means that the key store is already enrolled.

In [79]:
## Check if the keystore is enrolled
%! blhost $CONNECTION read-memory 0x0009E600 4

## Key provisioning enroll uncomment in case it's not enrolled ###
%! blhost $CONNECTION key-provisioning enroll
assert _exit_code == 0

### Write KEK for SB 
%! blhost $CONNECTION key-provisioning set_user_key 3 $SBKEK_BIN
assert _exit_code == 0

### Generate random key for PRINCE region 0 ###
%! blhost $CONNECTION key-provisioning set_key 7 16
assert _exit_code == 0

### Generate random key for PRINCE region 1 ###
%! blhost $CONNECTION key-provisioning set_key 8 16
assert _exit_code == 0

### Generate random key for PRINCE region 2 ###
%! blhost $CONNECTION key-provisioning set_key 9 16
assert _exit_code == 0

### In case the key store has been enrolled comment if enrolling
# read keystore from internal flash
# %! blhost $CONNECTION key-provisioning read_key_nonvolatile 0

# # Write KEK for SB 
# %! blhost $CONNECTION key-provisioning set_user_key 3 $SBKEK_BIN

# ### Now write the keystore to internal flash
%! blhost $CONNECTION key-provisioning write_key_nonvolatile 0
assert _exit_code == 0


blhost -u lpc55 read-memory 0x0009E600 4
Reading memory  [####################################]  100%
00 00 00 00
Response status = 0 (0x0) Success.
Response word 1 = 4 (0x4)
Read 4 of 4 bytes.
blhost -u lpc55 key-provisioning enroll
Response status = 0 (0x0) Success.
blhost -u lpc55 key-provisioning set_user_key 3 workspace/sbkek.bin
Response status = 0 (0x0) Success.
blhost -u lpc55 key-provisioning set_key 7 16
Response status = 0 (0x0) Success.
blhost -u lpc55 key-provisioning set_key 8 16
Response status = 0 (0x0) Success.
blhost -u lpc55 key-provisioning set_key 9 16
Response status = 0 (0x0) Success.
blhost -u lpc55 key-provisioning write_key_nonvolatile 0
Response status = 0 (0x0) Success.


### Receive SB file
Last step is to receive SB and then reset the chip to boot the image. Do not forget to remove the ISP jumper.

In [80]:
%! blhost $CONNECTION receive-sb-file $SB2_PATH
assert _exit_code == 0

# reset target
%! blhost $CONNECTION reset
assert _exit_code == 0

blhost -u lpc55 receive-sb-file workspace/output.sb2
Sending SB file  [####################################]  100%          
Response status = 0 (0x0) Success.
blhost -u lpc55 reset
Response status = 0 (0x0) Success.
